In [ ]:
# !python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_sm
# !pip install spacy[transformers]
# pip install fuzzywuzzy
# pip install python-Levenshtein
# !pip install openpyxl
# !pip install python-certifi-win32
# !pip install spacy
# pip install ruamel.yaml
# pip install pyarrow
#pip install swifter

In [1]:
import swifter
import spacy
import ssl
from spacy import displacy
import pandas as pd
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
from sklearn.metrics import accuracy_score

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import process, fuzz

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tqdm import tqdm
tqdm.pandas()

In [2]:
df = pd.read_parquet('C:/Users/Olimpio.Dasilva/Desktop/MACHINE LEARNING/DATA/reporting_ncr_all_transactions_202302131044.parquet')

NER = spacy.load("en_core_web_sm")


In [3]:
df.head()

,fi_id,account_id,transaction_id,amount,currency,txn_desc1,txn_desc2,region,transaction_timestamp,merchant_id,merchant_name,payment_agent,payment_agent_id,extract_date,filename,name
0,DI1004,Adb7cee870873375e9b71f97b49bb47f3,T8c3444845ea837d98f6492fbfeb996b1,300.0,USD,External Withdrawal TJMAXX,MASTERCAR NEEDHAM BANK BP BILL PAYMT,MT,2023-02-08,12897.0,TJ MAXX,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
1,DI1004,A0d0f7d433a6f3cf4815fc5f7f77bbf73,Td0c9a05199c034f282105c258edf4c90,42.5,USD,POS Withdrawal (FIP) SUNOCO,*63 SUNOCO *63 FRAMINGHAM MA(8631),None,2023-02-08,12696.0,SUNOCO,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
2,DI1004,Ac9fb869c66b83cb7a6f0d84e5f37d60a,Tf9ce6b86a4e1379eb600c9af4c1ae90e,250.0,USD,External Withdrawal BARCLAYCARD,US CREDITCARD,None,2023-02-07,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
3,DI1004,Ade85fffa5da73362bd6b0522721820a3,T4f1445aa5fd536f09d13bd6918c057fb,42.5,USD,ATM Withdrawal (FIP) SUNSHINE,LLC-60 464 MAIN ST WEST SPRINGFI MA(5681),None,2023-02-08,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank
4,DI1004,A1f3390a72bf035a48a21fce7d72df58a,T7be6870fc08e30c48fd78836885adf12,114.5,USD,POS Withdrawal (FIP) NNT,DESTINATION 735 PROVIDENCE HWY DEDHAM MA(7169),None,2023-02-11,NaN,Unknow Merchant,Unknow Payment Agent,NaN,2023-02-13,FTS_CSLK_TRANSAC_20230212_100011.csv,Needham Bank


In [4]:
print('Total number of lines and columns on the original dataset is: ', df.shape)

Total number of lines and columns on the original dataset is:  (2280287, 16)


1. Dataset with Unknow Merchants

In [5]:
df_unknown_merchs = df.copy(deep=True)
df_unknown_merchs.shape

(2280287, 16)

2. Dataset with Known Merchants

In [6]:
df_known_merchs = df_unknown_merchs.copy(deep=True)
df_known_merchs = df_known_merchs[df_unknown_merchs["merchant_name"].str.contains("Unknow Merchant") == False]
df_known_merchs.shape

(1133623, 16)

In [7]:
#Creating new columns CONCAT_COLS, AFS_MECHANT_NAMES, NER_RAW_MODEL
pd.set_option('display.max_colwidth', None)


df2 = df_known_merchs[['txn_desc1','txn_desc2','merchant_name']].copy(deep=True)
df2["CONCAT_COLS"] = df['txn_desc1'].astype(str) +" %%%%% "+ df['txn_desc2'].astype(str)

df2['NER_RAW_MODEL'] = pd.Series(dtype='string')

df2['AFS_MECHANT_NAMES']= df2['merchant_name'].str.lower().str.replace(' ','').fillna('-')

df2.shape

(1133623, 6)

2.1. NER model application

In [ ]:
def raw_model(text:str):
    text_ner = NER(text)
    r_orgs = []
    for word in text_ner.ents:
        if word.label_ == "ORG":
            r_orgs.append(word.text)
    return ' '.join(r_orgs) if r_orgs else 'NA'

In [ ]:
#df2['NER_RAW_MODEL'] = df2.CONCAT_COLS.progress_apply(raw_model)

df2['NER_RAW_MODEL'] = df2['CONCAT_COLS'].swifter.apply(raw_model)
df2.head(5)


In [8]:
df2['CONCAT_COLS']= df2['CONCAT_COLS'].str.lower().str.replace(r'\s{2,}','').str.replace(r"[,;@#?!&$*]+\ *", " ")
df2['NER_RAW_MODEL']= df2['NER_RAW_MODEL'].str.lower().str.replace(' ','').fillna('-')

df2.head(5)

,txn_desc1,txn_desc2,merchant_name,CONCAT_COLS,NER_RAW_MODEL,AFS_MECHANT_NAMES
0,External Withdrawal TJMAXX,MASTERCAR NEEDHAM BANK BP BILL PAYMT,TJ MAXX,external withdrawal tjmaxx %%%%% mastercar needham bank bp bill paymt,-,tjmaxx
1,POS Withdrawal (FIP) SUNOCO,*63 SUNOCO *63 FRAMINGHAM MA(8631),SUNOCO,pos withdrawal (fip) sunoco %%%%% 63 sunoco 63 framingham ma(8631),-,sunoco
5,POS Withdrawal (FIP) KOHLS 0575,1 WORCESTER RD FRAMINGHAM MA(8806),KOHL'S,pos withdrawal (fip) kohls 0575 %%%%% 1 worcester rd framingham ma(8806),-,kohl's
7,POS Withdrawal (FIS),AMAZON.COM*HE3H AMAZON.COM SEATTLE WA(6966),AMAZON.COM,pos withdrawal (fis) %%%%% amazon.com he3h amazon.com seattle wa(6966),-,amazon.com
12,POS Withdrawal (FIS),AMAZON.COM*HR0D AMAZON.COM SEATTLE WA(6966),AMAZON.COM,pos withdrawal (fis) %%%%% amazon.com hr0d amazon.com seattle wa(6966),-,amazon.com


In [9]:
df2.nunique()

txn_desc1            132917
txn_desc2            655042
merchant_name          1793
CONCAT_COLS          745566
NER_RAW_MODEL             1
AFS_MECHANT_NAMES      1793
dtype: int64

In [10]:
(df2.AFS_MECHANT_NAMES == df2.NER_RAW_MODEL).value_counts()

False    1133623
dtype: Int64

In [11]:
# Getting number of total merchant names
a = df2.AFS_MECHANT_NAMES
aunique = a.unique()
dist_list = pd.DataFrame(aunique)
print('The total number of distinct merchants is: ', len(dist_list))

The total number of distinct merchants is:  1793


In [12]:
df3 = df2[['CONCAT_COLS','AFS_MECHANT_NAMES', 'NER_RAW_MODEL']].copy(deep=True)

In [13]:
i=0
org = df3['AFS_MECHANT_NAMES']
txt = df3['CONCAT_COLS']
patterns1 =[]
org_list = org.tolist()
org_lis = []

for lis in dist_list: #org_list // if org_list[i] != 'unknowmerchant':
    org_lis.append(org_list[i])
    i+=1
    
patterns1.append([{"label": "ORG", "pattern": dist_list}]) #org_lis

#print(patterns1)

#Counts number of distinct merchants in the list
uniqueList = []
count1=0
for merchant in org_lis:
    if merchant not in uniqueList:
        uniqueList.append(merchant)
    count1 +=1
    


#print(uniqueList , count1)


In [14]:
from spacy.matcher import Matcher
from spacy.pipeline import EntityRuler

#Build upon the spaCy Model
NER = spacy.load("en_core_web_sm")

org = df3['AFS_MECHANT_NAMES']
#df3['PRED_2'].notna()


#Create the EntityRuler
ruler = NER.add_pipe("entity_ruler", config={'validate': True})


#List of Entities and Patterns
org_list = org.tolist()
org_lis = []

i=0
for lis in org_list: 
    org_lis.append(org_list[i]) 
    i+=1
    
a=0
patterns = []
for lis in range(len(org_list)): #range(500)
    patterns.append({"label": "ORG", "pattern": org_lis[a]}) #org_lis // dist_list
    a+=1
    

#print(patterns)
ruler.add_patterns(patterns)

c:\Users\Olimpio.Dasilva\Anaconda3\lib\site-packages\spacy\pipeline\entityruler.py:339: UserWarning: [W012] A Doc object you're adding to the PhraseMatcher for pattern 'ORG' is parsed and/or tagged, but to match on 'ORTH', you don't actually need this information. This means that creating the patterns is potentially much slower, because all pipeline components are applied. To only create tokenized Doc objects, try using `nlp.make_doc(text)` or process all texts as a stream using `list(nlp.tokenizer.pipe(all_texts))`.
  self.phrase_matcher.add(label, [pattern])  # type: ignore


In [18]:
def find_org(text:str):
    text_ner = NER(text)
    orgs = []
    for word in text_ner.ents:
        if word.label_ == "ORG":
            orgs.append(word.text)
    #return orgs if orgs else 'NA'
    return ' '.join(orgs) if orgs else 'NA'

In [17]:
df3['PREDICTIONS'] = df3.CONCAT_COLS.swifter.apply(find_org)

Pandas Apply:   0%|          | 0/1133623 [00:00<?, ?it/s]

In [19]:
df3.PREDICTIONS

0                              tjmaxx
1                   fip sunoco sunoco
5                                 fip
7               amazon.com amazon.com
12         amazon.com hr0d amazon.com
                      ...            
2280281                     starbucks
2280282                            NA
2280283                    7-eleven %
2280285                            NA
2280286                       express
Name: PREDICTIONS, Length: 1133623, dtype: object

In [20]:
## to check ###

from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from fuzzywuzzy import process, fuzz

def check_names(a, b):
    if ((a in b) or (b in a)):
        return True
    elif (fuzz.ratio(a, b) > 90):
        return True
    else:
        return False



In [21]:
df3.head(5)

,CONCAT_COLS,AFS_MECHANT_NAMES,NER_RAW_MODEL,PREDICTIONS
0,external withdrawal tjmaxx %%%%% mastercar needham bank bp bill paymt,tjmaxx,-,tjmaxx
1,pos withdrawal (fip) sunoco %%%%% 63 sunoco 63 framingham ma(8631),sunoco,-,fip sunoco sunoco
5,pos withdrawal (fip) kohls 0575 %%%%% 1 worcester rd framingham ma(8806),kohl's,-,fip
7,pos withdrawal (fis) %%%%% amazon.com he3h amazon.com seattle wa(6966),amazon.com,-,amazon.com amazon.com
12,pos withdrawal (fis) %%%%% amazon.com hr0d amazon.com seattle wa(6966),amazon.com,-,amazon.com hr0d amazon.com


In [27]:
#New column CHECK for True/False results

df3['CHECK'] = df3.apply(lambda x: check_names(x.AFS_MECHANT_NAMES, x.PREDICTIONS), axis=1)
df3.head(10)

,CONCAT_COLS,AFS_MECHANT_NAMES,NER_RAW_MODEL,PREDICTIONS,CHECK,SIMILARITY
0,external withdrawal tjmaxx %%%%% mastercar needham bank bp bill paymt,tjmaxx,-,tjmaxx,True,100
1,pos withdrawal (fip) sunoco %%%%% 63 sunoco 63 framingham ma(8631),sunoco,-,fip sunoco sunoco,True,52
5,pos withdrawal (fip) kohls 0575 %%%%% 1 worcester rd framingham ma(8806),kohl's,-,fip,False,0
7,pos withdrawal (fis) %%%%% amazon.com he3h amazon.com seattle wa(6966),amazon.com,-,amazon.com amazon.com,True,65
12,pos withdrawal (fis) %%%%% amazon.com hr0d amazon.com seattle wa(6966),amazon.com,-,amazon.com hr0d amazon.com,True,56
13,pos withdrawal (fis) ebay %%%%% o 23-09685 ebay o 23-09685-754 san jose ca(2810),ebay,-,ebay ebay ca(2810,True,38
14,pos withdrawal (fip) nws %%%%% staples 000 881 worcester road natick ma(8806),staples,-,fip staples,True,78
18,pos withdrawal (fis) nike.com %%%%% nike.com 6645 ca(9293),nike,-,NA,False,0
19,pos withdrawal (fis) www costco %%%%% com www costco com 800-955-2292 wa(9702),costco,-,costco costco,True,63
23,pos withdrawal (fip) star %%%%% market 36 star market 36 boston ma(1516),starmarket,-,fip,False,0


In [28]:
# Checking Accuracy
print('Accuracy: {0:0.9f}'.format(accuracy_score(df3.PREDICTIONS, df3.AFS_MECHANT_NAMES)))


Accuracy: 0.290023226


In [29]:

df3['SIMILARITY'] = pd.Series(dtype='string')

df3['SIMILARITY'] = df3.apply(lambda x: fuzz.ratio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

#df3['SIMILARITY'] = df3.apply(lambda x: fuzz.token_sort_ratio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

#df3['SIMILARITY'] = df3.apply(lambda x: fuzz.WRatio(x['AFS_MECHANT_NAMES'], x['PREDICTIONS']), axis=1)

df3.head(10)

,CONCAT_COLS,AFS_MECHANT_NAMES,NER_RAW_MODEL,PREDICTIONS,CHECK,SIMILARITY
0,external withdrawal tjmaxx %%%%% mastercar needham bank bp bill paymt,tjmaxx,-,tjmaxx,True,100
1,pos withdrawal (fip) sunoco %%%%% 63 sunoco 63 framingham ma(8631),sunoco,-,fip sunoco sunoco,True,52
5,pos withdrawal (fip) kohls 0575 %%%%% 1 worcester rd framingham ma(8806),kohl's,-,fip,False,0
7,pos withdrawal (fis) %%%%% amazon.com he3h amazon.com seattle wa(6966),amazon.com,-,amazon.com amazon.com,True,65
12,pos withdrawal (fis) %%%%% amazon.com hr0d amazon.com seattle wa(6966),amazon.com,-,amazon.com hr0d amazon.com,True,56
13,pos withdrawal (fis) ebay %%%%% o 23-09685 ebay o 23-09685-754 san jose ca(2810),ebay,-,ebay ebay ca(2810,True,38
14,pos withdrawal (fip) nws %%%%% staples 000 881 worcester road natick ma(8806),staples,-,fip staples,True,78
18,pos withdrawal (fis) nike.com %%%%% nike.com 6645 ca(9293),nike,-,NA,False,0
19,pos withdrawal (fis) www costco %%%%% com www costco com 800-955-2292 wa(9702),costco,-,costco costco,True,63
23,pos withdrawal (fip) star %%%%% market 36 star market 36 boston ma(1516),starmarket,-,fip,False,0


In [32]:
(df3.AFS_MECHANT_NAMES == df3.PREDICTIONS).value_counts()

False    804846
True     328777
dtype: int64

In [33]:
df3.shape

(1133623, 6)